# Extractor

In [ ]:
import pdfplumber
import pandas as pd
import re
from tqdm import tqdm

# --- CONFIGURAZIONE ---
pdf_path = "Other/EUCS PDF official file/EUCS – Cloud Service candidate cybersecurity certification scheme.pdf"
output_csv = "Other/EUCS PDF official file/EUCS_AnnexA_Extraction_Clean.csv"

# Range pagine (Verifica che l'Annex A sia qui)
START_PAGE = 30
END_PAGE = 200

def clean_text(text):
    if not text: return ""
    # Rimuove newline, caratteri non-breaking space e spazi multipli
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')
    return re.sub(r'\s+', ' ', text).strip()

def is_control_id(val):
    """Verifica se una stringa sembra un ID di controllo (es. OIS-01.1)"""
    if not val: return False
    # Regex: Lettere maiuscole, trattino, numeri, punto, numeri
    return bool(re.search(r'[A-Z]{2,4}-\d{2}(\.\d)?', str(val)))

extracted_data = []

print(f"Avvio estrazione tabelle da pag. {START_PAGE} a {END_PAGE}...")

with pdfplumber.open(pdf_path) as pdf:
    # Seleziona le pagine
    pages = pdf.pages[START_PAGE:END_PAGE]
    
    for page in tqdm(pages, desc="Processing"):
        # Estrae le tabelle dalla pagina
        tables = page.extract_tables()
        
        for table in tables:
            for row in table:
                # Pulizia base della riga (rimuove None e spazi)
                cleaned_row = [clean_text(cell) for cell in row]
                
                # --- LOGICA DI FILTRO ---
                
                # 1. Salta righe vuote
                if not any(cleaned_row): 
                    continue
                
                # Uniamo tutto il testo della riga per cercare parole chiave
                row_full_text = " ".join(cleaned_row).lower()
                
                # 2. SALTA RIGHE DI INTESTAZIONE o GUIDANCE
                if "control id" in row_full_text or "requirement" in row_full_text:
                    continue 
                
                if "guidance" in row_full_text or "implementation" in row_full_text:
                    continue 
                
                # --- LOGICA DI ESTRAZIONE CONTROLLO ---
                
                # Cerchiamo l'ID nella prima colonna utile
                potential_id = cleaned_row[0]
                
                if is_control_id(potential_id):
                    # Abbiamo trovato un ID valido!
                    
                    # --- FIX ANTI-CRASH ---
                    # Invece di cleaned_row[1], prendiamo tutto ciò che c'è dopo l'indice 0.
                    # Se la lista ha solo 1 elemento, questo restituisce stringa vuota "" invece di crashare.
                    description = " ".join(cleaned_row[1:]) 
                    
                    # Cerchiamo il livello (Basic/Substantial/High)
                    assurance_level = "Unknown"
                    
                    # Proviamo a vedere se l'ultima colonna è un livello esplicito
                    if len(cleaned_row) >= 3:
                        last_col = cleaned_row[-1]
                        if any(x in last_col for x in ["Basic", "Substantial", "High"]):
                            assurance_level = last_col
                            # Rimuoviamo il livello dalla descrizione se è stato incluso per sbaglio
                            description = description.replace(last_col, "").strip()
                    
                    # Se "Unknown", cerchiamo nel testo della riga
                    if assurance_level == "Unknown":
                        lev_match = re.search(r'(Basic|Substantial|High)', row_full_text, re.IGNORECASE)
                        if lev_match:
                            assurance_level = lev_match.group(1)

                    extracted_data.append({
                        "controlId": potential_id,
                        "description": description,
                        "assurance_level": assurance_level.capitalize()
                    })

# --- SALVATAGGIO ---
df = pd.DataFrame(extracted_data)

# Rimuovi duplicati
df = df.drop_duplicates(subset=['controlId'])

# Filtro qualità: Rimuovi righe con descrizioni troppo corte (probabili errori di parsing)
df = df[df['description'].str.len() > 5]

# Creiamo il baseId per compatibilità
df['baseId'] = df['controlId'].apply(lambda x: x.split('.')[0] if '.' in x else x)

print(f"\nEstrazione completata: {len(df)} controlli trovati.")
df.to_csv(output_csv, index=False)
print(f"File salvato in: {output_csv}")

# Anteprima
print(df.head())

## Cleaner

In [ ]:
import pandas as pd
import re

# --- CONFIGURAZIONE ---
input_csv = "Other/EUCS PDF official file/EUCS_AnnexA_Extraction_Clean.csv"
output_csv = "Other/EUCS PDF official file/EUCS_AnnexA_Extraction_Perfect.csv"

print("--- AVVIO PULIZIA AVANZATA ---")
df = pd.read_csv(input_csv)

# 1. FIX TYPOS NEGLI ID (Es. OSI -> OIS)
print("1. Correzione Typos negli ID...")
# Lista di correzioni note nel PDF ENISA
corrections = {
    'OSI-': 'OIS-',
    'IAM- ': 'IAM-',
    'OPS- ': 'OPS-'
}

def fix_id_typos(val):
    val = str(val).strip()
    for wrong, right in corrections.items():
        if val.startswith(wrong):
            return val.replace(wrong, right)
    return val

df['controlId'] = df['controlId'].apply(fix_id_typos)
# Ricalcoliamo il baseId per sicurezza
df['baseId'] = df['controlId'].apply(lambda x: x.split('.')[0] if '.' in x else x)


# 2. PULIZIA DESCRIZIONE (Rimozione Livelli e Caratteri Strani)
print("2. Pulizia Descrizioni...")

def clean_description(row):
    text = str(row['description'])
    
    # A. Rimuovi caratteri bullet point del PDF
    text = text.replace('', '-').replace('•', '-')
    
    # B. Rimuovi il livello se è finito in coda al testo (Es. "...security. Basic")
    # Cerchiamo " Basic", " Substantial", " High" alla fine della stringa
    # Regex: Spazio + (Basic|Substantial|High) + Fine stringa
    text = re.sub(r'\s+(Basic|Substantial|High)\s*$', '', text, flags=re.IGNORECASE)
    
    # C. Se la descrizione contiene SOLO il livello (errore di parsing), svuotala
    if text.strip().lower() in ['basic', 'substantial', 'high']:
        return ""
        
    return text.strip()

df['description'] = df.apply(clean_description, axis=1)


# 3. FIX LIVELLI MANCANTI ("Unknown")
print("3. Recupero Livelli mancanti...")
# Se il livello è Unknown, proviamo a dedurlo se era rimasto nel testo originale (prima della pulizia)
# Nota: Dato che abbiamo già pulito la descrizione, usiamo una logica euristica inversa o manuale se necessario.
# Ma spesso il parser lo aveva messo in "description" e noi l'abbiamo tolto.
# Per semplicità, qui impostiamo un default o verifichiamo se possiamo fare meglio.

# (Opzionale) Rimuovi righe vuote o inutili
initial_len = len(df)
df = df[df['description'].str.len() > 10]
print(f"   - Rimosse {initial_len - len(df)} righe troppo corte/sporche.")

# --- SALVATAGGIO ---
df.to_csv(output_csv, index=False)

print("-" * 30)
print(f"✅ FILE PULITO SALVATO: {output_csv}")
print("-" * 30)

# Verifiche specifiche
print("🔍 Verifica Correzione OSI-01.4:")
check = df[df['controlId'].str.contains("01.4")]
if not check.empty:
    print(check[['controlId', 'baseId']].to_string(index=False))

--- AVVIO PULIZIA AVANZATA ---
1. Correzione Typos negli ID...
2. Pulizia Descrizioni...
3. Recupero Livelli mancanti...
   - Rimosse 0 righe troppo corte/sporche.
------------------------------
✅ FILE PULITO SALVATO: Other/EUCS PDF official file/EUCS_AnnexA_Extraction_Perfect.csv
------------------------------
🔍 Verifica Correzione OSI-01.4:
controlId baseId
 OIS-01.4 OIS-01
  HR-01.4  HR-01
  AM-01.4  AM-01
  CS-01.4  CS-01
 DEV-01.4 DEV-01
  PM-01.4  PM-01
  IM-01.4  IM-01
  CO-01.4  CO-01
 PSS-01.4 PSS-01
